In [1]:
# импортируем необходимые библиотеки, классы и функции
import numpy as np  
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [2]:
# загружаем данные
data = pd.read_csv('Data/bankloan.csv', sep=';', decimal=',')

# разбиваем данные на обучающие и тестовые: 
# получаем обучающий массив предикторов, 
# тестовый массив предикторов, обучающий
# массив меток, тестовый массив меток
X_train, X_test, y_train, y_test = train_test_split(
    data.drop('default', axis=1), 
    data['default'], 
    test_size=0.3,
    stratify=data['default'],
    random_state=42)

# создаем список количественных переменных
num_cols = ['age', 'debtinc', 'creddebt', 'othdebt']

# создаем экземпляр класса StandardScaler
standardscaler = StandardScaler()

# выполняем стандартизацию
standardscaler.fit(X_train[num_cols])
X_train[num_cols] = standardscaler.transform(X_train[num_cols])
X_test[num_cols] = standardscaler.transform(X_test[num_cols])

# выполняем дамми-кодирование
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

In [3]:
# пишем собственный класс, строящий логистическую
# регрессию с регуляризацией и использованием
# градиентного спуска
class RegularizedLogisticRegression_GD:
    """
    Класс, строящий логистическую регрессию
    с регуляризацией и использованием
    градиентного спуска.
    
    Параметры
    ----------
    penalty: string, по умолчанию 'l2'
        Метод регуляризации. 
        Можно выбрать 'l1' или 'l2'.
    lr: float, по умолчанию 0.1
        Темп обучения.
    tol: float, по умолчанию 1e-5
        Допуск сходимости.
    max_iter: int, по умолчанию 1e7
        Максимальное количество итераций 
        градиентного спуска.
    lambda_: float, по умолчанию 0.001
        Сила регуляризации.
    fit_intercept: bool, по умолчанию True
        Добавление константы.   
    """
    def __init__(self, penalty='l2', lr=0.1, tol=1e-5, max_iter=1e7,
                 lambda_=0.001, fit_intercept=True):
        
        # проверяем параметр penalty, задающий регуляризацию,
        # на соответствие значениям 'l1' или 'l2'
        if penalty not in ['l2', 'l1']:
            raise ValueError(
                "penalty must be 'l1' or 'l2' "
                "got '%s' instead" % penalty
            )
        
        # тип регуляризации (должен быть либо 'l1' , либо 'l2')
        self.penalty = penalty
        # темп обучения
        self.lr = lr
        # допуск сходимости
        self.tol = tol
        # максимальное количество итераций
        self.max_iter = max_iter
        #  штрафной коэффициент, т.е. вводим штраф за слишком
        # большие оценки коэффициентов регрессии
        self.lambda_ = lambda_
        # добавление константы
        self.fit_intercept = fit_intercept

    # метод .fit() выполняет обучение
    def fit(self, X, y):
        # если задан параметр fit_intercept=True
        if self.fit_intercept:
            # то добавляем константу, т.е. добавляем
            # первый столбец из единиц
            X = np.c_[np.ones(X.shape[0]), X]

        # инициализируем значение функции потерь на предыдущей
        # итерации бесконечно большим значением
        l_prev = np.inf
        # инициализируем веса признаков нулями
        self.beta = np.zeros(X.shape[1])

        # выполняем градиентный спуск
        for _ in range(int(self.max_iter)):
            # применяем сигмоид-преобразование к скалярному произведению
            # массива предикторов и вектора весов, получаем
            # вероятности положительного класса
            y_pred = self._sigmoid(np.dot(X, self.beta))
            # вычисляем значение логистической функции потерь
            loss = self._NLL(X, y, y_pred)
            # если разница между предыдущим значением и текущим значением 
            # функции потерь меньше заданного порога (tol), то прерываем 
            # цикл, т.е. реализована ранняя остановка, которая ограничивает 
            # количество итераций (max_iter)
            if l_prev - loss < self.tol:
                return
            # присваиваем функции потерь на предыдущей 
            # итерации текущее значение
            l_prev = loss
            # обновляем веса, вычитаем из текущего приближения вектора 
            # весов вектор градиента, умноженный на некоторый 
            # темп обучения
            self.beta -= self.lr * self._NLL_grad(X, y, y_pred)

    # метод _sigmoid вычисляет значение сигмоиды
    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    # метод _NLL вычисляет значение логистической функции потерь
    def _NLL(self, X, y, y_pred):
        # вычисляем значение логистической функции потерь без штрафа
        nll = -np.log(np.where(y == 1, y_pred, 1 - y_pred)).sum()
        
        # вычисляем штрафное слагаемое, представляющее собой
        # произведение штрафного коэффициента и L2-нормы весов 
        # (регрессионных коэффициентов), если order=2, возвращаем 
        # np.sum(np.abs(x)**2)**(1./2), т.е. квадратный корень из 
        # суммы квадратов модулей регрессионных коэффициентов и 
        # возводим в квадрат, чтобы получить сумму квадратов 
        # регрессионных коэффициентов
        if self.penalty == 'l2': 
            # если первый элемент вектора весов - это константа,
            # регуляризацию применяем ко всем элементам вектора
            # весов, кроме первого
            if self.fit_intercept:
                penalty = (self.lambda_ / 2) * np.linalg.norm(
                    self.beta[1:], ord=2) ** 2             
            # если вектор весов не содержит константу, применяем
            # регуляризацию ко всем элементам вектора весов
            else:
                penalty = (self.lambda_ / 2) * np.linalg.norm(
                    self.beta, ord=2) ** 2
        
        # вычисляем штрафное слагаемое, представляющее собой 
        # произведение штрафного коэффициента и L1-нормы весов 
        # (регрессионных коэффициентов), если order=1, возвращаем 
        # np.sum(np.abs(x)), т.е. сумму модулей регрессионных
        # коэффициентов
        if self.penalty == 'l1':
            # если первый элемент вектора весов - это константа,
            # регуляризацию применяем ко всем элементам вектора
            # весов, кроме первого
            if self.fit_intercept:
                penalty = self.lambda_ * np.linalg.norm(
                    self.beta[1:], ord=1)
            # если вектор весов не содержит константу, применяем
            # регуляризацию ко всем элементам вектора весов
            else:
                penalty = self.lambda_ * np.linalg.norm(
                    self.beta, ord=1)
            
        # вычисляем итоговое значение логистической функции потерь, 
        # прибавив к значению логистической функции потерь штрафное 
        # слагаемое, полученную сумму делим на количество наблюдений
        return (penalty + nll) / X.shape[0]

    # метод _NLL_grad вычисляет вектор градиента
    def _NLL_grad(self, X, y, y_pred):
        # если тип регуляризации l2
        if self.penalty == 'l2':
            # если первый элемент вектора весов - это константа,
            # регуляризацию применяем ко всем элементам вектора
            # весов, кроме первого
            if self.fit_intercept:
                # штрафуем все веса, кроме первого (константы)
                d_penalty = self.lambda_ * self.beta[1:]
                # подставляем константу в вектор оштрафованных весов
                d_penalty = np.r_[self.beta[0], d_penalty]
            # если вектор весов не содержит константу, применяем
            # регуляризацию ко всем элементам вектора весов
            else:
                d_penalty = self.lambda_ * self.beta

        # если тип регуляризации l1
        if self.penalty == 'l1':
            # если первый элемент вектора весов - это константа,
            # регуляризацию применяем ко всем элементам вектора
            # весов, кроме первого
            if self.fit_intercept:
                # штрафуем все веса, кроме первого (константы)
                d_penalty = self.lambda_ * np.sign(self.beta[1:])
                # подставляем константу в вектор оштрафованных весов
                d_penalty = np.r_[self.beta[0], d_penalty]
            # если вектор весов не содержит константу, применяем
            # регуляризацию ко всем элементам вектора весов
            else:
                d_penalty = self.lambda_ * np.sign(self.beta)
                
        # получаем вектор градиента, для этого вычисляем скалярное 
        # произведение матрицы предикторов и вектора разностей между 
        # вероятностями положительного класса и фактическими значениями
        # зависимой переменной, прибавляем к полученным результатам 
        # оштрафованные веса, берем итоги и делим на количество 
        # наблюдений
        return (np.dot(y_pred - y, X) + d_penalty) / X.shape[0]

    # метод .predict_proba() вычисляет вероятности
    def predict_proba(self, X):
        # если задано fit_intercept=True
        if self.fit_intercept:
            # добавляем константу
            X = np.c_[np.ones(X.shape[0]), X]
        return self._sigmoid(np.dot(X, self.beta))

    # метод .predict() вычисляет прогнозы
    def predict(self, X, threshold):
        # получаем прогнозы в зависимости от установленного порога
        return (self.predict_proba(X) >= threshold).astype(int)

In [4]:
%%time

# создаем экземпляр нашего класса
# RegularizedLogisticRegression_GD
model = RegularizedLogisticRegression_GD()
# обучаем модель логистической регрессии
model.fit(X_train, y_train)
# вычислим AUC-ROC
proba = model.predict_proba(X_test)
print("AUC-ROC на тестовой выборке: %.3f" % roc_auc_score(y_test, proba))

AUC-ROC на тестовой выборке: 0.788
CPU times: user 930 ms, sys: 188 ms, total: 1.12 s
Wall time: 142 ms


In [5]:
# создаем игрушечный массив предикторов
X_toy = np.array([[0.1, 0.2, 0.3], 
                  [0.7, 0.5, 0.2],
                  [0.2, 0.4, 1.4],
                  [0.4, 0.1, 0.5]])

# создаем игрушечный массив значений
# зависимой переменной
y_toy = np.array([0, 1, 0, 0])

In [6]:
# создаем вектор весов, инициализируем веса очень 
# небольшими положительными значениями, близкими к 0
beta = np.array([0.01, 0.012, 0.015])

In [7]:
# пишем функцию сигмоид-преобразования
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [8]:
# к скалярному произведению матрицы предикторов и вектора весов
# применяем сигмоид-преобразование и получаем вероятности 
# положительного класса
y_pred = sigmoid(np.dot(X_toy, beta))
y_pred

array([0.50197499, 0.50399991, 0.50694955, 0.50317496])

In [9]:
# задаем тип регуляризации
penalty = 'l2'

In [10]:
# вычисляем значение логистической функции потерь без штрафа
nll = -np.log(np.where(y_toy == 1, y_pred, 1 - y_pred)).sum()
nll

2.7889452861319945

In [11]:
# задаем значение штрафного коэффициента
lambda_ = 0.1

In [12]:
# убеждаемся, что с помощью np.linalg.norm() возвращаем 
# сумму квадратов регрессионных коэффициентов
print(np.linalg.norm(beta, ord=2) ** 2)
print(np.sum(beta ** 2))

0.00046899999999999996
0.000469


In [13]:
# убеждаемся, что с помощью np.linalg.norm() возвращаем
# сумму модулей регрессионных коэффициентов
print(np.linalg.norm(beta, ord=1))
print(np.sum(np.abs(beta)))

0.037
0.037


In [14]:
# вычисляем штрафное слагаемое, представляющее собой
# произведение штрафного коэффициента и L2-нормы весов 
# (регрессионных коэффициентов), если order=2, возвращаем 
# np.sum(np.abs(x)**2)**(1./2), т.е. квадратный корень из 
# суммы квадратов модулей регрессионных коэффициентов и 
# возводим в квадрат, чтобы получить сумму квадратов 
# регрессионных коэффициентов
penalty_term = (lambda_ / 2) * np.linalg.norm(beta, ord=2) ** 2
penalty_term

2.345e-05

In [15]:
# вычисляем значение логистической функции потерь 
# со штрафом, прибавив к значению логистической 
# функции потерь штрафное слагаемое, полученную
# сумму делим на количество наблюдений
loss = (penalty_term + nll) / X_toy.shape[0]
loss

0.6972421840329986

In [16]:
# получаем оштрафованные веса, для этого
# умножаем веса на штрафной коэффициент
d_penalty = lambda_ * beta
d_penalty

array([0.001 , 0.0012, 0.0015])

In [17]:
# получаем вектор градиента
NLL_grad = (np.dot(y_pred - y_toy, X_toy) + d_penalty) / X_toy.shape[0]
NLL_grad

array([0.00166433, 0.02667307, 0.25355233])

In [18]:
# задаем темп обучения
lr = 0.1

# обновляем веса, вычитаем из текущего приближения вектора весов 
# вектор градиента, умноженный на некоторый темп обучения
beta = beta - lr * NLL_grad
beta

array([ 0.00983357,  0.00933269, -0.01035523])

In [19]:
# пишем класс RegularizedLogisticRegression_Newton, 
# строим логистическую регрессию с L2-регуляризацией и
# использованием ньютоновского метода
class RegularizedLogisticRegression_Newton():
    """
    Класс, строящий логистическую регрессию
    с L2-регуляризацией и использованием
    метода Ньютона.
    
    Параметры
    ----------
    tol: float, по умолчанию 1e-5
        Допуск сходимости.
    max_iter: int, по умолчанию 10
        Максимальное количество итераций 
        ньютоновской оптимизации.
    lambda_: float, по умолчанию 0.001
        Сила регуляризации.
    fit_intercept: bool, по умолчанию True
        Добавление константы.   
    """
    def __init__(self, tol=1e-5, max_iter=10, lambda_=0.001, 
                 fit_intercept=True):
        # максимальное количество итераций
        self.max_iter = max_iter
        # допуск сходимости
        self.tol = tol
        #  штрафной коэффициент, т.е. вводим штраф за слишком
        # большие оценки коэффициентов регресии
        self.lambda_ = lambda_
        # добавление константы
        self.fit_intercept = fit_intercept

    # метод .fit() выполняет обучение
    def fit(self, X, y):
        # если задан параметр fit_intercept=True
        if self.fit_intercept:
            # то добавляем константу, т.е. добавляем
            # первый столбец из единиц
            X = np.c_[np.ones(X.shape[0]), X]
            
        # инициализируем значение функции потерь на предыдущей
        # итерации бесконечно большим значением
        l_prev = np.inf
        # инициализируем веса признаков нулями
        self.beta = np.zeros(X.shape[1])

        # выполняем ньютоновскую оптимизацию
        for _ in range(int(self.max_iter)):
            # применяем сигмоид-преобразование к скалярному произведению
            # массива предикторов и вектора весов, по сути получаем
            # вероятности положительного класса
            y_pred = self._sigmoid(np.dot(X, self.beta))
            
            # вычисляем значение логистической функции потерь
            loss = self._NLL(X, y, y_pred)
            # если разница между предыдущим значением и текущим значением 
            # функции потерь меньше заданного порога (tol), то прерываем 
            # цикл, т.е. реализована ранняя остановка, которая ограничивает 
            # количество итераций (max_iter)
            if l_prev - loss < self.tol:
                return
            # присваиваем функции потерь на предыдущей 
            # итерации текущее значение
            l_prev = loss 
            # обновляем веса, вычитаем из текущего приближения вектора 
            # весов вектор градиента, умноженный на некоторый 
            # темп обучения
            self.beta -= self._NLL_grad(X, y, y_pred)

    # метод _sigmoid вычисляет значение сигмоиды
    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    # метод _NLL вычисляет значение логистической функции потерь
    def _NLL(self, X, y, y_pred):
        # вычисляем значение логистической функции потерь без штрафа
        eps = 1e-15
        y_pred = np.clip(y_pred, eps, 1 - eps)
        nll = -np.log(np.where(y == 1, y_pred, 1 - y_pred)).sum()
        
        # вычисляем штрафное слагаемое, представляющее собой
        # произведение штрафного коэффициента и L2-нормы весов 
        # (регрессионных коэффициентов), если order=2, возвращаем 
        # np.sum(np.abs(x)**2)**(1./2), т.е. квадратный корень из 
        # суммы квадратов модулей регрессионных коэффициентов и 
        # возводим в квадрат, чтобы получить сумму квадратов 
        # регрессионных коэффициентов
       
        # если первый элемент вектора весов - это константа,
        # регуляризацию применяем ко всем элементам вектора
        # весов, кроме первого
        if self.fit_intercept:
            penalty = (self.lambda_ / 2) * np.linalg.norm(
                self.beta[1:], ord=2) ** 2             
        # если вектор весов не содержит константу, применяем
        # регуляризацию ко всем элементам вектора весов
        else:
            penalty = (self.lambda_ / 2) * np.linalg.norm(
                self.beta, ord=2) ** 2
            
        # вычисляем итоговое значение логистической функции потерь, 
        # прибавив к значению логистической функции потерь штрафное 
        # слагаемое, полученную сумму делим на количество наблюдений
        return (penalty + nll) / X.shape[0]
    
    # метод _NLL_grad вычисляет вектор градиента
    def _NLL_grad(self, X, y, y_pred):      
        # если первый элемент вектора весов - это константа,
        # регуляризацию применяем ко всем элементам вектора
        # весов, кроме первого
        if self.fit_intercept:
            d_penalty = np.r_[self.beta[0], self.lambda_ * self.beta[1:]]
            lambda_diag = np.diag(np.r_[0, [self.lambda_] * len(
                self.beta[1:])])
        # если вектор весов не содержит константу, применяем
        # регуляризацию ко всем элементам вектора весов
        else:
            d_penalty = self.lambda_ * self.beta
            lambda_diag = np.diag(np.ones_like(self.beta) * self.lambda_)
        
        # получаем вектор градиента, для этого вычисляем скалярное 
        # произведение матрицы предикторов и вектора разностей между 
        # вероятностями положительного класса и фактическими значениями 
        # зависимой переменной, прибавляем к полученным результатам 
        # оштрафованные веса, берем итоги и делим на количество наблюдений
        grad = (np.dot(X.T, y_pred - y) + d_penalty) / X.shape[0]
        # вычисляем гессиан
        X_ = (y_pred * (1 - y_pred))[:, np.newaxis] * X
        hess = np.dot(X_.T, X) / X.shape[0] + lambda_diag  
        # получаем обратный гессиан
        inv_hess = np.linalg.pinv(hess)
        # возвращаем произведение градиента и обратного гессиана
        return np.dot(inv_hess, grad)
    
    # метод .predict_proba() вычисляет вероятности
    def predict_proba(self, X):
        # если задано fit_intercept=True
        if self.fit_intercept:
            # добавляем константу
            X = np.c_[np.ones(X.shape[0]), X]
        return self._sigmoid(np.dot(X, self.beta))

    # метод .predict() вычисляет прогнозы
    def predict(self, X, threshold):
        # получаем прогнозы в зависимости от установленного порога
        return (self.predict_proba(X) >= threshold).astype(int)

In [20]:
%%time

# создаем экземпляр нашего класса
model = RegularizedLogisticRegression_Newton()
# обучаем модель логистической регрессии
model.fit(X_train, y_train)
# вычислим AUC-ROC
proba = model.predict_proba(X_test)
print("AUC-ROC на тестовой выборке: %.3f" % roc_auc_score(y_test, proba))

AUC-ROC на тестовой выборке: 0.786
CPU times: user 30.9 ms, sys: 23.2 ms, total: 54.1 ms
Wall time: 7.27 ms
